# redis笔记整理

## 1.描述

redis(Remote Dictionary Server):非关系型数据库，以键值对的形式进行数据存储（key-value）

存储在内存中 ***性能高***  Redis能读的速度是110000次/s,写的速度是81000次/s

redis与mongodb的比较

1.都是基于c++实现的

2.都依赖于内存，但Redis的TPS（吞吐量）比MongoDB高

    TPS=并发数/平均响应时间
    并发数：系统同时处理的request/事务数
    响应时间：一般取平均响应时间

3.mongodb提供了丰富的数据表达式，接近SQL,而Redis提供较少的IO

4.mongodb 不支持事务，而Redis支持事务，但只保证事务按顺序执行

5.mongodb支持Maprrduce(内部数据分析)，redis不支持

6.海量数据处理Mongdb优于redis

***特性***

数据的持久化：可以将内存中的数据保存在磁盘中，重启的时候可以再次加载进行使用。

丰富的数据类型：不仅仅支持简单的key-value类型的数据，同时还提供list，set，zset，hash等数据结构的存储。

数据的备份：即master-slave模式的数据备份。


## 2.安装

sudo apt install redis-server

sudo service redis-server status

sudo service redis-server restart

ps -ef | grep redis  # 查看redis进程

netstat -antp | grep 6379  # 查看redis服务占用的端口

## 3.进入redis环境

redis-cli  进入redis环境

redis-cli -h 127.0.0.1 -p 6379  #指定ip地址和端口

-r n 代表将命令重复执行n次

-i n 每隔n秒执行一次命令，必须与-r同时使用

redis-cli -r 3 -i 0.1

redis-cli -r 5 -i 1 info | grep used_memory_human

每隔1秒查询内存使用概况，一共查询5次

-a <passwd>  指定密码登录  redis-cli get passwd  查看密码

redis-cli info cpu  查看redis服务的cpu使用情况

exit/quit


## 4.redis配置文件

配置文件路径

/etc/redis/redis.conf

sudo vi /etc/redis/redis.conf

***功能***

设置密码

    修改requirepass为登录密码
    
    requirepass disen888   设置密码
    
    重启服务  sudo service redis-server restart
    
    使用口令登录
    
    redis-cli -a disen888
    
绑定主机地址
    
    修改bind的127.0.0.1为1的本地实际ip地址
    
    重启服务：sudo service redis-server restart
    
    指定ip连接redis: redis-cli -h 10.35.163.5
    
保护模式

    redis4.0+ 版本保护模式
    
    bind 127.0.0.1:: 只限本地连接
    
    protected-mode yes  开启保护模式
    

## 5.基本操作

### 1.设置键值 获取键的值  删除键

set name "lili"  # 用set指令设置key value

get name      # 获取name的值

del name      # 删除name键

### 2.key操作的命令

exists key # 确认一个key是否存在  1为真，0为假

type key  # 返回key值的类型

keys pattern  # 返回满足给定pattern的所有key 如keys *name*

randomkey  # 随机返回一个存在的key

rename oldname newname  # 重命名key

dbsize  # 返回当前数据库中key的数目

exprie keyname seconds  # 获得一个key的活动时间(s)（这个key为当前库中存在的key，设置时间后，当时间终止时，该key会自动消失）

ttl keyname # 获得一个key的活动时间，当返回的结果为(integer) -1时，代表可以一直活动

select 1 # 切换到1数据库中

move keyname dbindex 移动当前数据库中的key到dbindex数据库

flushdb  # 删除当前选择数据库中的所有key

flushall # 删除所有数据库中的所有key

### 3.string 操作命令

getset key value   # 将给定key的值设为value，并返回key的旧值(old value)

mget key1 key2 [key3……]   # 获取一个或多个指定key的值

setnx key value  # 添加新key的value,如果key已经存在，则什么都不做

setex key time value   # 修改key的value，并设置过期时间

mset key value [key value ……]  # 批量设置多个string的值

msetnx key value [key value……]  # 添加多个都不存在的key-value

incr key #将key的value增1，只对数字的value递增

incrby key integer  # 将key的valuez增加指定的数值

desc key  # 将key的value减1，只对数字的value进行递减

descby key integer  # 将key的value减去指定的数值

append key value  # 附加value，相对于Python中字符串的拼接

substr key start end  返回value的子串（与Python中的列表的切片类似，唯一不同的是end包含在内 sex "maledsf"  substr sex 0 3  返回结果为male ）


### 4.list操作命令

rpush key value [value……] # 在名称为key的list尾添加一个值为value的元素（也可以创建list类型的value）

lpush key value [value ...]  # 在名称为key的list头添加一个值为value的元素(从左侧传的第一个值被最后一个返回出) lpush age 1 2 3 4 5 返回结果为5 4 3 2 1

llen key  返回名称为key的list的长度

***lrange key start end*** 返回名称为key的list中start之end之间的元素（start位置的元素与end位置的元素都会显示）

lrange key 0 -1  显示所有元素


ltrim key start end 截取名称为key的list,截取之后在查询，结果为截取之后的内容

lindex key index  返回名称为key的list中index位置的元素

lset key index value 给名称为key的list中的元素赋值

lrem key count value  删除count个key的list中值为value的元素

lpop key  返回并删除名称为key的list的首元素

rpop key 返回并删除名称为key的list的尾元素


### 5.set操作

sadd key member  # 向名称为key的set中添加元素member(也可以创建set)

srem key member  # 删除名称为key的set中的元素member

spop key      # 随机返回并删除名称为key的set中的一个元素

smove srckey dstkey member # 移到集合元素（将member元素从srckey集合移动到dstkey集合）

scard key 返回名称为key的set的个数

sismember key member  # member是否为名称为key的set中的元素，返回结果为1或0

sinter key1 key2 key3……  求交集

sinterstore dstkey key1 key2……   求key1 key2的交集，并将交集保存在dstkey中(可添加多个集合)

sunion  key1  key2 [key3 ...]         求并集
 
sunionstore  dstkey  key1 [key2 ...]    求并集并将并集保存到dstkey的集合

sdiff  key1 key2 [key3……]          求差集

a 1 2 3

b 1 3 5

sdiff a b  2

sdiff b a 5

sdiffstore  dstkey  key1 key2 [key3 ...]  求差集并将差集保存到dstkey的集合

smembers key  返回名称为key的set的所有元素

srandmerber key  随机返回名称为key的set的一个元素


### 6.哈希操作的命令

hset key field value #向名称为key的hash中添加元素

hget key  field  #返回名称为key的hash中field对应的value
 
hmget key fields   # 返回名称为key的hash中field i对应的value

hmset key fields   # 向名称为key的hash中添加元素field

hincrby key  field  integer  将名称为key的hash中field的value增加integer

hexists key  field   名称为key的hash中是否存在键为field的域

hdel key  field  删除名称为key的hash中键为field的域

hlen key   返回名称为key的hash中元素个数

hkeys key  返回名称为key的hash中所有键
 
hvals key  返回名称为key的hash中所有键对应的value

hgetall key  返回名称为key的hash中所有的键 field 及其对应的value



### 7.切换数据库

redis数据库以索引号标识，默认为0号库

select 1 切换到1号库

keys * 查看1号库下所有的keys

## 6.python 连接redis

安装驱动 pip3 install redis

pip3 install python-redis

***Python代码连接***

import redis

连接redis  

db = redis.Redis()  #默认host=127.0.0.1 port=6379  decode_responses=False

db = redis.StrictRedis()

数据操作

db.set('name','disen')  添加string类型的key-value

print(db.get('name')) 读取string类型的key

db.lpush('name','disen','jack','rose')

print(db.lrange('name',0,-1))

db.save()